In [20]:
import os
import sys

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import binary_crossentropy

# 0. Data Load

In [3]:
train = pd.read_json(os.path.join('..', 'Data', 'train.json'))
test = pd.read_json(os.path.join('..', 'Data', 'test.json'))

In [4]:
train.head()

,user_note,rating_per_user,vintage_id,user_like_count,userID,wine_id,wine_name,url,like
0,,4.0,142514980,0,19484511,2532733,Secateurs Red Blendn2016,/badenhorst-family-wines-secateurs-red-blend-s...,1
1,,3.5,48346487,0,19484511,1253802,Kloof Street Swartland Rougen2015,/mullineux-kloof-street-swartland-rouge/w/1253...,0
2,Beautiful golden nectar,3.5,156865950,1,19484511,1123441,Late Harvestn2018,/stables-late-harvest/w/1123441?year=2018,0
3,,3.5,150284640,0,19484511,1157656,Proseccon2017,/ruffino-prosecco/w/1157656?year=2017,0
4,Bright clean and refreshing,3.5,1471106,0,19484511,1134756,Prosecco TrevisonN.V.,/la-gioiosa-prosecco-treviso/w/1134756?year=N.V.,0


# 1. Data Process

In [5]:
train = train[['userID', 'wine_id', 'like']]
test = test[['userID', 'wine_id', 'like']]

In [6]:
user_ids = train['userID'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

wine_ids = train['wine_id'].unique().tolist()
wine2wine_encoded = {x: i for i, x in enumerate(wine_ids)}
wine_encoded2wine = {i: x for i, x in enumerate(wine_ids)}

In [7]:
add_train = pd.DataFrame(zip(train['userID'].map(user2user_encoded), train["wine_id"].map(wine2wine_encoded), train['like']),
             columns = ['userID', 'wine_id', 'like'])

add_test = pd.DataFrame(zip(test['userID'].map(user2user_encoded), test["wine_id"].map(wine2wine_encoded), test['like']),
             columns = ['userID', 'wine_id', 'like'])

In [8]:
add_train

,userID,wine_id,like
0,0,0,1
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
...,...,...,...
937751,6342,4672,0
937752,6342,1023,0
937753,6342,14075,1
937754,6342,3894,0


In [9]:
X = add_train[['userID', 'wine_id']].values
y = add_train['like'].values

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

# 2. Model

In [11]:
sys.path.append(os.path.join('..', 'Models'))
import NeuralMF

In [12]:
num_users = len(user2user_encoded)
num_itmes = len(wine_encoded2wine)

EMBEDDING_SIZE = 50

In [21]:
model = NeuralMF.RecommenderNet(num_users, num_itmes, EMBEDDING_SIZE)

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(), 
    optimizer = keras.optimizers.Adam(lr = 0.001),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(
            name='binary_accuracy', dtype = None, threshold = 0.5)
    ])

# 3. Train

In [22]:
callbacks_list = [
                  keras.callbacks.EarlyStopping(
                      monitor = 'binary_accuracy',
                      patience = 5
                      )]

In [23]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 100,
    verbose = 1,
    validation_data = (x_val, y_val),
)

Epoch 1/100
5861/5861 [==============================] - 84s 14ms/step - loss: 0.6826 - binary_accuracy: 0.5661 - val_loss: 0.6791 - val_binary_accuracy: 0.5889
Epoch 2/100
5861/5861 [==============================] - 83s 14ms/step - loss: 0.7048 - binary_accuracy: 0.5807 - val_loss: 0.6963 - val_binary_accuracy: 0.5972
Epoch 3/100
5861/5861 [==============================] - 82s 14ms/step - loss: 0.7591 - binary_accuracy: 0.5825 - val_loss: 0.7269 - val_binary_accuracy: 0.6033
Epoch 4/100
5861/5861 [==============================] - 82s 14ms/step - loss: 0.8264 - binary_accuracy: 0.5899 - val_loss: 0.7638 - val_binary_accuracy: 0.5959
Epoch 5/100
5861/5861 [==============================] - 83s 14ms/step - loss: 1.0271 - binary_accuracy: 0.5571 - val_loss: 0.9233 - val_binary_accuracy: 0.5837
Epoch 6/100
5861/5861 [==============================] - 82s 14ms/step - loss: 1.4248 - binary_accuracy: 0.5605 - val_loss: 1.1497 - val_binary_accuracy: 0.5248
Epoch 7/100
5861/5861 [===========